In [ ]:
import numpy as np
import pandas as pd

def getDF(pc, bl):
    parent_kids = pd.read_csv(pc, delimiter =',', names = ['Parent', 'Child']) # load up the parent child table
    branches = pd.DataFrame({"Lengths": list(open(bl))[0].split(',')}) # load up the branch lengths
    combined = pd.concat([parent_kids, branches], axis = 1) # combine parent child table + branch lengths 

    n = min(parent_kids.loc[:,'Parent']) # smallest parent
    l = range(n-1,0, -1) #list of numbers from the min-1 to 1

    combined = pd.concat([combined, pd.DataFrame({"Sequences" : ["NA"]*len(combined["Parent"])})], axis = 1) # add NA for sequences

    sequences = pd.read_csv("/Users/cmco/Desktop/APP/msa.dat", header = None, delimiter = " ") # load up sequences for terminal nodes
    sequences = sequences.iloc[::-1] # order from highest to lowest
    sequences.columns =["Parent", "Lengths"] # give column names

    # combine sequences, branch lengths for terminal nodes
    kids = pd.DataFrame({"Parent" : list(l), "Child" : ["NA"]*len(l), "Lengths" : ["NA"]*len(l), "Sequences" : sequences["Lengths"]})

    # return whole combined df
    return pd.concat([combined, kids], ignore_index = True, axis = 0)

tree = getDF("/Users/cmco/Desktop/APP/table.dat", "/Users/cmco/Desktop/APP/branchlength.dat")
print(tree)



   Parent  Child Lengths Sequences
0       9      1     0.1        NA
1       9      2     0.4        NA
2       8      9    0.01        NA
3       8      3    0.04        NA
4       7      4     0.2        NA
5       7      5    0.08        NA
6       6      7    0.12        NA
7       6      8    0.14        NA
   Parent Child Lengths                       Sequences
4       5    NA      NA  CGCTACCGCTACCGCTACCGCTACCGCTAC
3       4    NA      NA  CGTTACCGTTACCGTTACCGTTACCGTTAC
2       3    NA      NA  CGCTATCGCTATCGCTATCGCTATCGCTAT
1       2    NA      NA  AGCTCAAGCTCAAGCTCAAGCTCAAGCTCA
0       1    NA      NA  AGATCAAGATCAAGATCAAGATCAAGATCA
    Parent Child Lengths                       Sequences
0        9     1     0.1                              NA
1        9     2     0.4                              NA
2        8     9    0.01                              NA
3        8     3    0.04                              NA
4        7     4     0.2                              NA
5      